# 🦾 Cyberwolf 🐺

**Let's play Werewolf with LLMs!**

Werewolf is a social deception game, where the participants are divided into Werewolves and Villages.

The villagers' goal is to lynch all the werewolves, while the werewolves' goal is to gain majority in the village.

During the day, Werewolves turn human, and try to stay undetected while the village discusses and votes on who to lynch. But during the night, they turn into blood-thirsty monsters! They then decide who to kill, and the next day, the villagers wake up to find one of their own dead.

When humans play it, everyone sits in a circle and closes their eyes. The werewolves then wake up (open their eyes) and decide who to kill (by pointing at them) before falling back asleep. Afterward, during the day, everyone wakes up and the game master tells them who was mauled during the night.

This is a game that is verbal in nature, and where deception and strategy come into play. So what happens when we get AI to play it against each other? Let's find out!

In [4]:
# @title Select the LLM model you want to use. { display-mode: "form" }
import os

local_models = ['llama-2-13b-chat.Q4_0', 'llama-2-13b-chat.Q5_K_M']
llm_model_option = 'llama-2-13b-chat.Q5_K_M' # @param ["llama-2-13b-chat.Q4_0", "llama-2-13b-chat.Q5_K_M", "gpt-4-1106-preview"]



model_is_local = llm_model_option in local_models

if model_is_local:
    os.environ["MODEL_IS_LOCAL"] = "1"
else:
    os.environ["MODEL_IS_LOCAL"] = "0"

## If using Llama - Set up the GPU

To start off, let's install and configure a large language model. This example runs in the free version of Google Colab with a T4 GPU.

We will use llama-index to power the model and llama2 as our model.

We first download llama-index via pip (it doesn't come pre-installed).

In [5]:
%%shell
if [ "$MODEL_IS_LOCAL" = "1" ]
then
    pip install llama-index wurlitzer
fi

We then compile it to use CUDA (make sure that you use a T4 GPU). You will need about 12GB of VRAM.

This step might take a minute.

In [6]:
%%shell
if [ "$MODEL_IS_LOCAL" = "1" ]
then
    CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
fi

We will download a LLAMA-2 model directly from HuggingFace.

In [7]:
if model_is_local:
    model_url = f"https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/{llm_model_option}.gguf"

In [8]:
if model_is_local:
    from llama_index import (
        SimpleDirectoryReader,
        VectorStoreIndex,
        ServiceContext,
    )
    from llama_index.llms import LlamaCPP
    from llama_index.llms.llama_utils import (
        messages_to_prompt,
        completion_to_prompt,
    )

Before we continue, we will also install wurlitzer. This is because we want to be able to capture C++'s stderr sys pipe when we load LlamaCPP - otherwise we cannot see if the runner picks up CUDA!

In [9]:
if model_is_local:
    from wurlitzer import pipes, sys_pipes

Now let's load the model. It is important that you transfer all model layers to the GPU (`n_gpu_layers` below) - otherwise, each steps takes minutes to compute.

You should also see something like

```
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
```

in the logs. This confirms that you are now running on CUDA.

Note - you cannot set `verbose=False` right now - see bug https://github.com/abetlen/llama-cpp-python/issues/729

In [10]:
if model_is_local:
    with sys_pipes():
        llm = LlamaCPP(
            # You can pass in the URL to a GGML model to download it automatically
            model_url=model_url,
            # optionally, you can set the path to a pre-downloaded model instead of model_url
            model_path=None,
            temperature=2.7,
            max_new_tokens=256,
            # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
            context_window=3900,
            # kwargs to pass to __call__()
            generate_kwargs={},
            # kwargs to pass to __init__()
            # set to at least 1 to use GPU
            model_kwargs={"n_gpu_layers": 43},
            # transform inputs into Llama2 format
            messages_to_prompt=messages_to_prompt,
            completion_to_prompt=completion_to_prompt,
            verbose=True,
      )

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - ten

In [11]:
if model_is_local:
    def call_llm(instructions: str, prompt: str):
        response = llm.chat(
            messages=[
                ChatMessage(
                    role=MessageRole.SYSTEM,
                    content=instructions
                ),
                ChatMessage(
                    role=MessageRole.USER,
                    content=prompt
                ),
            ]
        )
        return response.message.content.strip()

# If using OpenAI - enter your API token

In [9]:
%%shell
if [ "$MODEL_IS_LOCAL" = "0" ]
then
    pip install openai python-dotenv
fi

In [10]:
if not model_is_local:
    from getpass import getpass
    from dotenv import load_dotenv
    import openai

    openai_api_key = getpass('Enter the OpenAI API Key: ')

    with open('.env', 'w') as f:
        f.write(f'OPENAI_API_KEY={openai_api_key}')

    load_dotenv()

    def call_llm(instructions: str, prompt: str):
        response = openai.chat.completions.create(
            model=llm_model_option,
            max_tokens=50,
            messages=[
                ChatMessage(
                    role=MessageRole.SYSTEM,
                    content=instructions
                ),
                ChatMessage(
                    role=MessageRole.USER,
                    content=prompt
                ),
            ]
        )
        response = response.choices[0].message.content.lstrip()
        return response

# Now let's start *playing*

## Define the game

### Import dependencies

First, let us import a number of things that we will use later!

In [12]:
import random
import collections
import numpy as np
from pydantic import BaseModel
from typing import *
from collections import defaultdict
import os
import sys
from llama_index.llms.base import ChatMessage, MessageRole

### Set up helpers

Now, let us define two helper functions. They each take instructions and a prompt, and then return a response. The response is either a choice (for voting) or a free text reponse, for the other players.

In [13]:
def choose(instructions: str, prompt: str, choices: list[str]):
    formatted_choices = '\n'.join(f'  - {choice}' for choice in choices)
    instruction_stem = f"""
RESPOND ONLY WITH ONE OF THE FOLLOWING CHOICES:
{formatted_choices}
"""
    instructions += instruction_stem
    prompt += instruction_stem

    response_choice = call_llm(instructions, prompt)
    if response_choice not in choices:
        for choice in choices:
            if choice in response_choice:
                response_choice = choice
                break
        else:
            print("Invalid choice:", response_choice)
            return None
    return response_choice

In [14]:
def prompt_with_instructions(instructions: str, prompt: str):
    response = call_llm(instructions, prompt)
    return response

### Configure the game

In here we will define the roles, names of players and other aspects.

We choose to keep it simple, with villages, werewolves and seers.

Feel free to configure the number of players, roles and other aspects of the game here.

In [15]:
VILLAGER = "Villager"
WEREWOLF = "Werewolf"
SEER = "Seer"
ROLE_NAMES = [
    VILLAGER,
    WEREWOLF,
    SEER,
]
ROLE_TYPE = Literal[tuple(ROLE_NAMES)]

UTTERANCES = 5
TABLE_CARDS = 2
ROLE_COUNTS = {
    VILLAGER: 7,
    WEREWOLF: 3,
    SEER: 2,
}
ALLOW_SEER_VOTE_SELF = False
ALLOW_WEREWOLVES_VOTE_WEREWOLF = False
ALLOW_LYNCH_VOTE_SELF = False

PLAYER_NAMES = ["Alice", "Bob", "Jim", "Friderik", "Janez", "Doris", "Paella", "Kiki", "Chiko", "Lima", "Romeo", "Juliet"]
SYSTEM = "System"
NAME_TYPE = Literal[tuple(PLAYER_NAMES)]

if sum(ROLE_COUNTS.values()) - TABLE_CARDS > len(PLAYER_NAMES):
    raise RuntimeError

In [16]:
class Message(BaseModel):
    name: NAME_TYPE | Literal[SYSTEM]
    message: str

### The player class

Now, let us define the player class. Broadly speaking the player can `vote` and `speak` at different points in the game. If the player gets mauled or lynched, they are dead (and thus silent) for the rest of the game.

In [17]:
class Player:
    def __init__(
        self,
        name: NAME_TYPE,
        role: ROLE_TYPE,
        use_selfstream: bool=False
    ):
        self.name = name
        self.role = role
        self.visibility = {}
        self.history: list[Message] = []
        self.instructions_suffix = ""

        self.alive = True
        self._use_self_stream: bool
        self._selfstream: list[str] = []

        self._use_selfstream = use_selfstream

        self._side = WEREWOLF + 's' if role == WEREWOLF else VILLAGER + 's'

    @property
    def uses_selfstream(self):
      return self._use_selfstream

    @property
    def selfstream(self):
      return self._selfstream[-1]


    def build_instructions(self):
        instructions = ("You are playing the game werewolf with several other players. "
            "The goal of the werewolves is to eat the villagers at night without being caught during the day. "
            "The goal of the villagers is to figure out who the werewolves are during the day by talking, and then voting them out of the village. "
            "The villagers win if they vote all the werewolves out. "
            "The werewolves win if they get majority. \n\n"
            f"There are {ROLE_COUNTS[VILLAGER]} villagers, {ROLE_COUNTS[WEREWOLF]} werewolves, {ROLE_COUNTS[SEER]} seers, and {TABLE_CARDS} roles that are face-down on the table. "
            f"\n\nYour name is {self.name}, your role is {self.role}. You want {self._side} to win!"
        )

        formatted_visibility = '\n'.join(f' - {name}: {role}' for name, role in self.visibility.items())
        if formatted_visibility:
            instructions += "You know the following roles:\n" + formatted_visibility
        if self.instructions_suffix:
            instructions += f'\n{self.instructions_suffix}'

        return instructions

    def _prepare_history(self):
        formatted_history = "\n\n".join(
            f"{msg.name}: {msg.message}"
            for msg in self.history
        )
        history = f"Conversation history:\n```\n{formatted_history}\n```"
        return history

    def _prepare_selfstream(self, round_info):
        instructions_stem = f"Share your internal thoughts. How can your side, the {self._side} win the game?"

        instructions = self.build_instructions() + '\n' + instructions_stem



        previous_thoughts = f"Previous thoughts: {self._selfstream[-1]}" if len(self._selfstream) > 0 else ""


        history = self._prepare_history()
        insights_prompt = (f"{history}\n\n"
                           f"Round info: {round_info}\n"
                           f"{previous_thoughts}\n"
                           f"{instructions_stem}"
        )

        return instructions, insights_prompt


    def _prepare_speech_without_selfstream(self, round_info):
        instructions_stem = f"BE SHORT AND SUCCINCT, REPLY ONLY WITH {self.name}'s RESPONSE. ONLY PROVIDE NEW INSIGHTS."
        if self.role == WEREWOLF:
            instructions_stem += "DON'T REVEAL THAT YOU ARE A WEREWOLF! PROTECT THE OTHER WEREWOLVES!"

        instructions = self.build_instructions() + '\n' + instructions_stem

        history = self._prepare_history()

        prompt = (f"{history}\n\n"
                  f"Round info: {round_info}\n"
                  f"{instructions_stem}\n"
                  "EXAMPLE_RESPONSE: `Let's figure out who the werewolf is!`\n\n"
                  f"As {self.name} who is a {self.role} I say:"
        )

        return instructions, prompt

    def _prepare_speech_with_selfstream(self):
        instructions_stem = f"IT IS YOUR TURN TO SPEAK. BE SHORT AND SUCCINCT, REPLY ONLY WITH {self.name}'s RESPONSE."

        instructions = self.build_instructions() + '\n' + instructions_stem

        prompt = ("Your internal thoughts (do not share these):\n"
                  f"{self._selfstream[-1]}\n\n"
                  f"```{instructions_stem}```\n\n"
                  "\nEXAMPLE_RESPONSE: `Let's figure out who the werewolf is!`\n\n"
                  f"As {self.name} who is a {self.role} I say:"
        )

        return instructions, prompt

    def _prepare_vote(
        self,
        prompt_suffix: str,
    ):
        instructions = self.build_instructions()

        history = self._prepare_history()
        prompt = (f"{history}\n\n"
                  f"{prompt_suffix}"
        )
        return instructions, prompt


    def speak(self, round_info):
        if self._use_selfstream:
            instructions, prompt = self._prepare_selfstream(round_info)
            self._selfstream.append(prompt_with_instructions(instructions, prompt))
            instructions, prompt = self._prepare_speech_with_selfstream()
            response = prompt_with_instructions(instructions, prompt)
        else:
            instructions, prompt = self._prepare_speech_without_selfstream(round_info)
            response = prompt_with_instructions(instructions, prompt)
        return response

    def vote(
        self,
        player_names: list[NAME_TYPE],
        prompt_suffix: str,
    ):
        instructions, prompt = self._prepare_vote(prompt_suffix)
        result = choose(instructions, prompt, player_names)
        if result is None:
            print(f"{self.name} didn't vote properly")
        return result

    def kill(self):
        self.alive = False

### The round

Now let us define the game itself. It consists of Rounds owned by the Round class.

A round supports the `prepare` and `run` commands.

In [18]:
class Round:
    def __init__(
        self,
        players: list[Player],
        table: list[ROLE_TYPE],
        print_selfstream: bool=True
    ):
        self.all_players = players
        self.table = table
        self.conversation_history: list[Message] = []
        self.roles: dict[NAME_TYPE, ROLE_TYPE] = {
            p.name: p.role
            for p in players
        }
        self.players_by_name = {
            p.name: p
            for p in players
        }
        self._print_selfstream = print_selfstream

    @property
    def players(self):
        return [p for p in self.all_players if p.alive]

    @property
    def dead_player_names(self):
        return [player.name for player in self.all_players if not player.alive]

    def round_info_message(self, round):
        dead_players = ','.join(self.dead_player_names)
        message = (f"This is round {round}. The players {dead_players} are dead.")

    def prepare(self, player: Player):
        if player.role == VILLAGER:
            player.instructions_suffix = f"Figure out who the werewolf is, then vote for them. "
        elif player.role == WEREWOLF:
            player.visibility |= {
                player.name: WEREWOLF
                for player in self.all_players
                if player.role == WEREWOLF
            }
            player.instructions_suffix = f"Hide your identity as werewolf and blame another villager for the deaths. "
        elif player.role == SEER:
            # TODO let them choose instead of random
            choices = [
                player.name for player in self.all_players
            ]
            choices += [
                f"Table card {i}" for i in range(len(self.table))
            ]
            choice = random.choice(choices)
            if choice.startswith("Table card "):
                i = int(choice.removeprefix("Table card "))
                # table
                player.visibility |= {
                    choice: self.table[i]
                }
            else:
                chosen_player = self.players_by_name[choice]
                player.visibility |= {
                    choice: chosen_player.role
                }
        else:
            assert_never(player)
            raise RuntimeError

    def act_at_night(self, role: ROLE_TYPE) -> Optional[str]:
        if role == VILLAGER:
            raise RuntimeError
        elif role == WEREWOLF:
            werewolves = [
                player for player in self.players
                if player.role == WEREWOLF
            ]
            votes = defaultdict(int)

            print(f"\n-------------- \n")

            print("The werewolves have awoken")
            for werewolf in werewolves:
                if not ALLOW_WEREWOLVES_VOTE_WEREWOLF:
                    choices = [
                        p.name for p in self.players
                        if p.role != WEREWOLF
                    ]
                else:
                    choices = [p.name for p in self.players]
                chosen_player = werewolf.vote(
                    choices,
                    f"It's night-time. As a werewolf, you must vote for who to kill. Who will you vote for, {werewolf.name}?"
                )
                if chosen_player is None:
                    continue
                print(f"{werewolf.name} voted for {chosen_player}")
                votes[chosen_player] += 1
            max_votes = max(votes.values())
            tie_players = [player for player, num_votes in votes.items() if num_votes == max_votes]

            if len(tie_players) > 1:
                print("The werewolves tied between the following players:", tie_players)
            else:
                print("They voted for:", tie_players[0])
            chosen_player = tie_players[0]

            player = self.players_by_name[chosen_player]
            player.kill()
            return f"{player.name} was found mauled in their own home. Must have been the werewolves!"
        elif role == SEER:
            choices = [
                player.name for player in self.all_players
            ]
            choices += [
                f"Table role {i}" for i in range(len(self.table))
            ]
            seers = [
                player for player in self.players
                if player.role == SEER
            ]
            for seer in seers:
                if not ALLOW_SEER_VOTE_SELF:
                    seer_choices = list(
                        set(choices) - {seer.name}
                    )
                else:
                    seer_choices = choices
                choice = seer.vote(
                    seer_choices,
                    f"It's night-time. As a seer, you may look at one of the player's roles, or one of the roles on the table face-down. Which one will you look at, {seer.name}? Be concise and provide a name only."
                )
                if choice is None:
                    continue
                if choice.startswith("Table role "):
                    i = int(choice.removeprefix("Table role "))
                    result = self.table[i]
                else:
                    result = self.players_by_name[choice].role
                seer.visibility |= {
                    choice: result
                }
                msg = f"As the seer, you have chosen to see {choice}, you saw {result}"
                print(f"{seer.name}: {msg}")
                seer.history.append(
                    Message(
                        name=SYSTEM,
                        message=msg,
                    )
                )
            return None
        else:
            assert_never(role)
            raise RuntimeError

    def run(self, end_round=10, start_round=0):
        print("Roles:")
        for player in self.all_players:
            print(f"  - {player.name}: {player.role}")

        # Setup (giving info to players on who is who)
        for player in self.all_players:
            self.prepare(player)

        round_num = start_round
        while round_num < end_round:
            # Night phase
            results = []
            for role in [WEREWOLF, SEER]:
                if role == WEREWOLF and round_num == 0:
                    continue
                result = self.act_at_night(role)
                if result:
                    results.append(result)
            for result in results:
                message = Message(
                    name=SYSTEM,
                    message=result,
                )
                print(result)
                for player in self.players:
                    player.history.append(message)

            # Discussion phase
            last_speaker = None
            for _ in range(UTTERANCES):
                available_players = set(self.players)
                if last_speaker is not None:
                    available_players -= {last_speaker}
                chosen_player = random.choice(list(available_players))

                round_info = self.round_info_message(round_num)
                response = chosen_player.speak(round_info)

                if self._print_selfstream and chosen_player.uses_selfstream:
                    print(f"[{chosen_player.name} thinks]: {chosen_player.selfstream}")

                print(f"\n[{chosen_player.name}]:\n {response}\n")
                print(f"-------------- \n")

                message = Message(
                    name=chosen_player.name,
                    message=response,
                )
                for player in self.players:
                    player.history.append(message)

                last_speaker = chosen_player

            # Voting phase
            votes = defaultdict(int)
            vote_messages = []
            print("Time to lynch")
            for player in self.players:
                choices = [p.name for p in self.players]
                if not ALLOW_LYNCH_VOTE_SELF:
                    choices = list(
                        set(choices) - {player.name}
                    )
                voted_player = player.vote(
                    [
                        p.name for p in self.players
                        if p is not player
                    ],
                    f"Discussions are now closed. Your role is {player.role} and your side must win. Who are you going to vote for to kill, {player.name}? Be concise and provide a name only."
                )
                if voted_player is None:
                    continue

                msg = f"{player.name} voted for {voted_player}"
                print(msg)
                message = Message(
                    name=SYSTEM,
                    message=msg,
                )
                vote_messages.append(message)

                votes[voted_player] += 1

            for message in vote_messages:
                for player in self.players:
                    player.history.append(message)

            max_votes = max(votes.values())
            tie_players = [player for player, num_votes in votes.items() if num_votes == max_votes]

            if len(tie_players) > 1:
                msg = f"No one died, because there was a tie between the following players: {','.join(tie_players)}"
            else:
                msg = f"The people voted for {tie_players[0]}, they have been lynched. RIP"
                self.players_by_name[tie_players[0]].kill()

            # Check win condition
            werewolf_count = len([
                player for player in self.players
                if player.role == WEREWOLF
            ])
            if werewolf_count == 0:
                print("VILLAGERS WIN!")
                break
            if werewolf_count > len(self.players) // 2 or len(self.players) == werewolf_count:
                print("WEREWOLVES WIN!")
                break

            print(msg)
            message = Message(
                name=SYSTEM,
                message=msg,
            )
            for player in self.players:
                player.history.append(message)

            round_num += 1

### Instantiate the round

In here we hand out cards to players to give them roles and define our game.

In [19]:
players = []
names = PLAYER_NAMES[:]
roles = dict(ROLE_COUNTS)

table_cards = []
# pick out table cards
for _ in range(TABLE_CARDS):
    available_roles = [
        role for role, count in roles.items()
        if count > 1
    ]
    picked_role = random.choice(available_roles)
    roles[picked_role] -= 1
    table_cards.append(picked_role)

# assign roles
for role, count in roles.items():
    for _ in range(count):
        name = random.choice(names)
        names.remove(name)
        use_selfstream = role == WEREWOLF
        players.append(
            Player(
                name=name,
                role=role,
                use_selfstream=use_selfstream,
            )
        )

round = Round(
    players=players,
    table=table_cards,
)

### Filter Llama output (optional)

Earlier, we set verbose=True in our LlamaCPP model. Unfortunately, we cannot remove this due to a bug. So we will filter stderr to remove Llama output, and focus on the game.

In [20]:
import re
class Filter(object):
    def __init__(self, stream, re_pattern):
        self.stream = stream
        self.pattern = re.compile(re_pattern) if isinstance(re_pattern, str) else re_pattern
        self.triggered = False

    def __getattr__(self, attr_name):
        return getattr(self.stream, attr_name)

    def write(self, data):
        if data == '\n' and self.triggered:
            self.triggered = False
        else:
            if self.pattern.search(data) is None:
                self.stream.write(data)
                self.stream.flush()
            else:
                # caught bad pattern
                self.triggered = True

    def flush(self):
        self.stream.flush()

# example
sys.stderr = Filter(sys.stdout, r'Llama.generate: prefix-match hit')  # filter out any line which contains "Read -1" in it


## Run the game

Finally, let's unleash the werewolves!

In [21]:
##%%
round.run()

Roles:
  - Kiki: Villager
  - Janez: Villager
  - Bob: Villager
  - Paella: Villager
  - Juliet: Villager
  - Lima: Werewolf
  - Friderik: Werewolf
  - Romeo: Werewolf
  - Alice: Seer
  - Doris: Seer
Alice: As the seer, you have chosen to see Friderik, you saw Werewolf
Doris: As the seer, you have chosen to see Romeo, you saw Werewolf

[Juliet]:
 Okay! *looks around the group nervously* Do any of you have any idea who the werewolf might be? Have you noticed anything suspicious?

-------------- 


[Alice]:
 I concur with Juliet, I think it's crucial to investigate further and determine the identity of the werewolf. I recommend we start by questioning Friderik, given that I saw the Werewolf role when I peeked at him.

-------------- 


[Bob]:
 Sure, let's start by asking Friderik some questions. How about we ask him about his whereabouts last night during the full moon? Maybe he can shed some light on any suspicious activity.

-------------- 


[Juliet]:
 Great idea, Bob! *nods eagerly* 

# Final thoughts

Who won? The villagers? The Werewolves? Let us know!